# Prompt Chaining Workflow

In this notebook, we'll explore the prompt chaining workflow.
Using Kotlin and Claude via the LangChain4j library,
we'll implement a practical example that demonstrates how to break down complex tasks into manageable steps.

## What is prompt chaining?

Prompt chaining is a technique where we decompose a complex task into a sequence of simpler steps.
Each step involves an LLM call that processes the output from the previous step.
This creates a _"chain"_ of prompts that collectively solve the original problem.

![Prompt Chaining Workflow Diagram](image/prompt_chaining.svg)

### Key benefits:
- By focusing on smaller tasks, improved accuracy
- Each step is controlled and verified
- Using different prompts is optimized for different tasks

### When to use prompt chaining

Common use cases include:

- Content generation followed by translation
- Creating outlines before drafting full documents
- Data extraction and transformation processes

## Setting up environment

First, let's set up a Kotlin notebook with the necessary dependencies:

In [1]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("dev.langchain4j:langchain4j:1.0.0-beta2")
@file:DependsOn("dev.langchain4j:langchain4j-anthropic:1.0.0-beta2")

We'll need an API key for accessing Claude:

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY")

## Creating the LLM interface

Next, build a helper function to handle LLM calls:

In [3]:
import dev.langchain4j.data.message.SystemMessage.systemMessage
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.model.anthropic.AnthropicChatModel
import dev.langchain4j.model.anthropic.AnthropicChatModelName
import dev.langchain4j.model.chat.chat

suspend fun llmCall(
    prompt: String,
    systemPrompt: String? = null,
    model: AnthropicChatModelName = AnthropicChatModelName.CLAUDE_3_7_SONNET_20250219
): String {
    val client = AnthropicChatModel.builder()
        .apiKey(apiKey)
        .modelName(model)
        .maxTokens(4096)
        .temperature(0.1)
        .build()

    return withContext(Dispatchers.IO) {
        val response = client.chat {
            systemPrompt?.let { messages += systemMessage(it) }
            messages += userMessage(prompt)
        }
        response.aiMessage().text()
    }
}

## Implementing the `chain` function

The core of implementation is the `chain` function, which manages the flow of data our prompt sequence:

In [4]:
suspend fun chain(input: String, prompts: List<String>): String {
    var result = input
    prompts.forEachIndexed { index, prompt ->
        println("Step ${index + 1}")
        result = llmCall("$prompt\nInput: $result")
        println(result)
    }
    return result
}

This function:
- Takes an initial input and a list of prompts
- Iterates through each prompt in a sequence
- Passes the result of each step as input to the next
- Tracks progress through console output

## Creating prompt sequence

Now define the series of prompts that make up our chain.
Each prompt handles a specific subtask:

In [5]:
val prompts = listOf(
    // Step 1
    """
    Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth
    """,
    // Step 2
    """
    Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth
    """,
    // Step 3
    """
    Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction
    """,
    // Step 4
    """
    Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |
    """
)

## Testing chain

Let's test implementation on a sample quarterly report:

In [6]:
val report =
    """
    Q3 Performance Summary:
    Our customer satisfaction score rose to 92 points this quarter.
    Revenue grew by 45% compared to last year.
    Market share is now at 23% in our primary market.
    Customer churn decreased to 5% from 8%.
    New user acquisition cost is $43 per user.
    Product adoption rate increased to 78%.
    Employee satisfaction is at 87 points.
    Operating margin improved to 34%.
    """.trimIndent()

Finally, execute the prompt chain

In [7]:
runBlocking {
    println("Input text:")
    println(report)
    val formattedResult = chain(report, prompts)
    println(formattedResult)
}

Input text:
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
Step 1
92: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
$43: new user acquisition cost
78%: product adoption rate
87: employee satisfaction
34%: operating margin
Step 2
92%: customer satisfaction score
45%: revenue growth
23%: market share
5%: customer churn
$43: new user acquisition cost
78%: product adoption rate
87%: employee satisfaction
34%: operating margin
Step 3
92%: customer satisfaction score
87%: employee satisfaction
78%: product adoption rate
45%: revenue growth
34%: operating margin
23%: market share
$43: new user acquisition cost
5%: customer churn
S

## Conclusion

Prompt chaining offers a powerful and flexible approach to building AI agents.
By breaking complex tasks into manageable steps,
we can achieve higher accuracy and better control over the generation process.

This pattern is particularly valuable when developing applications that require structured data transformation,
multi-stage content generation, or any task that benefits from a divide-and-conquer approach.